In [490]:
import tensorflow as tf

## Task 1: Improving the accuracy of our logistic regression on MNIST

We got the accuracy of ~90% on our MNIST dataset with our vanilla model, which is
unacceptable. 

The dataset is basically solved and state of the art models <a href="http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html">reach accuracies above
99%</a> . 

You can use whatever loss functions, optimizers, even models that you want, as long as
your model is built in TensorFlow. 

You can save your code in the file q2.py. In the comments,
explain what you decided to do, instruction on how to run your code, and report your results.

I’m happy if you can reach 97%.

In [491]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Approach: Three-layer Neural Network

In [492]:
# Data Params
numTrainingSamples = mnist.train.images.shape[0] # 55000
numFeatures = mnist.train.images.shape[1] # 784
numClasses = mnist.train.labels.shape[1] # 10

# Network Params
numHiddenUnits1 = 200
numHiddenUnits2 = 200
numHiddenUnits3 = 200
numEpochs = 10000
batchSize = 200

In [493]:
X = tf.placeholder(tf.float32, shape=[None, numFeatures])
y = tf.placeholder(tf.float32, shape=[None, numClasses])

In [494]:
def layer_weight(inputDim, outputDim):
    return tf.Variable(tf.truncated_normal([inputDim, outputDim], stddev=0.1, seed=1))

def layer_bias(dim):
    return tf.Variable(tf.constant(0.1), [dim])

def network_layer(layerInput, weight, bias):
    layerOutput = tf.nn.relu(tf.matmul(layerInput, weight) + bias)
    return layerOutput

In [495]:
weights = {
    'hidden1': layer_weight(numFeatures, numHiddenUnits1),
    'hidden2': layer_weight(numHiddenUnits1, numHiddenUnits2),
    'hidden3': layer_weight(numHiddenUnits2, numHiddenUnits3),
    'output': layer_weight(numHiddenUnits3, numClasses)
}

biases = {
    'hidden1': layer_bias(numHiddenUnits1),
    'hidden2': layer_bias(numHiddenUnits2),
    'hidden3': layer_bias(numHiddenUnits3),
    'output': layer_bias(numClasses),
}

In [496]:
hiddenLayer1Output = network_layer(X, weights['hidden1'], biases['hidden1'])
hiddenLayer2Output = network_layer(hiddenLayer1Output, weights['hidden2'], biases['hidden2'])
hiddenLayer3Output = network_layer(hiddenLayer2Output, weights['hidden3'], biases['hidden3'])
finalOutput = network_layer(hiddenLayer3Output, weights['output'], biases['output'])

In [497]:
correct_prediction = tf.equal(tf.argmax(finalOutput,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [498]:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=y,
        logits=finalOutput
    )
)

optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

In [499]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(numEpochs):
    batch = mnist.train.next_batch(batch_size=batchSize)
    sess.run(optimizer, feed_dict={X: batch[0], y: batch[1]})
    if ((i+1)<10 or (i+1)%1000 == 0):
        batch_accuracy = accuracy.eval(feed_dict={X: batch[0], y: batch[1]})
        print("Iteration %s, training accuracy: %s" % ((i+1), batch_accuracy))

Iteration 1, training accuracy: 0.24
Iteration 2, training accuracy: 0.26
Iteration 3, training accuracy: 0.29
Iteration 4, training accuracy: 0.305
Iteration 5, training accuracy: 0.31
Iteration 6, training accuracy: 0.41
Iteration 7, training accuracy: 0.41
Iteration 8, training accuracy: 0.435
Iteration 9, training accuracy: 0.45
Iteration 1000, training accuracy: 0.97
Iteration 2000, training accuracy: 0.975
Iteration 3000, training accuracy: 0.99
Iteration 4000, training accuracy: 1.0
Iteration 5000, training accuracy: 0.995
Iteration 6000, training accuracy: 1.0
Iteration 7000, training accuracy: 0.995
Iteration 8000, training accuracy: 1.0
Iteration 9000, training accuracy: 1.0
Iteration 10000, training accuracy: 1.0


In [500]:
training_accuracy = accuracy.eval(feed_dict={
    X: mnist.train.images, 
    y: mnist.train.labels
})*100
testing_accuracy = accuracy.eval(feed_dict={
    X: mnist.test.images, 
    y: mnist.test.labels
})*100

predictions = sess.run(tf.argmax(
    tf.nn.softmax(finalOutput), 1
), feed_dict={X: mnist.test.images})

labels = sess.run(tf.argmax(mnist.test.labels, 1))

print("========================================================================")
print("===== Three-layer Neural Network with (%s, %s, %s) hidden units =====" % 
      (numHiddenUnits1, numHiddenUnits2, numHiddenUnits3)
)
print("========================================================================")
print("Training accuracy:\t %.3f %%" % training_accuracy)
print("Testing accuracy:\t %.3f %%" % testing_accuracy)

print("\nConfusion Matrix: \n")
print(sess.run(tf.confusion_matrix(labels=labels, predictions=predictions)))

===== Three-layer Neural Network with (200, 200, 200) hidden units =====
Training accuracy:	 99.762 %
Testing accuracy:	 97.740 %

Confusion Matrix: 

[[ 951    0    3    0    1    7   10    4    1    3]
 [   0 1122    3    2    0    1    2    1    3    1]
 [   0    0 1004    9    3    0    2    8    6    0]
 [   1    0    4  995    0    1    0    4    4    1]
 [   0    0    4    0  962    1    1    5    1    8]
 [   3    0    0   11    2  866    5    1    2    2]
 [   2    3    0    2    5    4  939    3    0    0]
 [   0    1    6    3    2    0    0 1012    1    3]
 [   2    0    3    5    1    4    2    4  949    4]
 [   1    2    1    9   10    1    1    7    3  974]]


## Task 2: Logistic regression on the notMNIST dataset

## Task 3: Logistic regression model on coronary heart disease dataset

In [523]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

In [524]:
BASE_DIR = "../../../"
DATA_DIR = os.path.join(BASE_DIR, "data/")

heart_disease = pd.read_csv(os.path.join(DATA_DIR, "heart.csv"))
heart_disease['famhist'] = heart_disease['famhist'].apply(lambda x: 1.0 if x=='Present' else 0.0)
heart_disease.head()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,1.0,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,0.0,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,1.0,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,1.0,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,1.0,60,25.99,57.34,49,1


In [525]:
heart_disease_data = heart_disease.as_matrix(
    ['sbp', 'tobacco', 'ldl', 'adiposity', 'famhist', 'typea', 'obesity', 'alcohol', 'age']
)
heart_disease_data = normalize(heart_disease_data)

heart_disease_target = np.array(
    [[0., 1.] if ele==1 else [1., 0.] for ele in heart_disease['chd']], 
    dtype="float32"
)

In [526]:
heart_disease_data[0]

array([ 0.78536336,  0.05890225,  0.02812583,  0.11343592,  0.00490852,
        0.24051753,  0.12418558,  0.47710824,  0.25524309])

In [527]:
print(heart_disease_data.shape, heart_disease_target.shape)

((462, 9), (462, 2))


In [528]:
X_train, X_test, y_train, y_test = train_test_split(
    heart_disease_data, 
    heart_disease_target, 
    train_size=0.8, 
    random_state=1
)

In [561]:
numFeatures = X_train.shape[1]
numClasses = y_train.shape[1]

numEpochs = 50000
learning_rate = 0.5

In [562]:
X = tf.placeholder(tf.float32, [None, numFeatures])
y = tf.placeholder(tf.float32, [None, numClasses])

In [563]:
W = tf.Variable(tf.truncated_normal([numFeatures, numClasses], stddev=0.1, seed=1))
b = tf.Variable(tf.constant(0.1), [numClasses])

In [564]:
y_prediction = tf.matmul(X, W) + b

In [565]:
correct_prediction = tf.equal(tf.argmax(y_prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [566]:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=y,
        logits=y_prediction
    )
)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [567]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(numEpochs):
    sess.run(optimizer, feed_dict={X: X_train, y: y_train})
    if ((i+1)<10 or (i+1)%5000==0):
        training_accuracy = accuracy.eval(feed_dict={X: X_train, y: y_train})
        print("Iteration %s, training accuracy: %s" % ((i+1), training_accuracy))

Iteration 1, training accuracy: 0.365854
Iteration 2, training accuracy: 0.639566
Iteration 3, training accuracy: 0.639566
Iteration 4, training accuracy: 0.639566
Iteration 5, training accuracy: 0.639566
Iteration 6, training accuracy: 0.639566
Iteration 7, training accuracy: 0.639566
Iteration 8, training accuracy: 0.639566
Iteration 9, training accuracy: 0.639566
Iteration 5000, training accuracy: 0.680217
Iteration 10000, training accuracy: 0.699187
Iteration 15000, training accuracy: 0.712737
Iteration 20000, training accuracy: 0.715447
Iteration 25000, training accuracy: 0.720867
Iteration 30000, training accuracy: 0.715447
Iteration 35000, training accuracy: 0.718157
Iteration 40000, training accuracy: 0.720867
Iteration 45000, training accuracy: 0.720867
Iteration 50000, training accuracy: 0.718157


In [568]:
training_accuracy = accuracy.eval(feed_dict={
    X: X_train, 
    y: y_train
})*100
testing_accuracy = accuracy.eval(feed_dict={
    X: X_test, 
    y: y_test
})*100

predictions = sess.run(tf.argmax(
    tf.nn.softmax(y_prediction), 1
), feed_dict={X: X_test})

labels = sess.run(tf.argmax(y_test, 1))

print("=================================================================")
print("===== Logistic Regression on coronary heart disease dataset =====")
print("=================================================================")
print("Training accuracy:\t %.3f %%" % training_accuracy)
print("Testing accuracy:\t %.3f %%" % testing_accuracy)

print("\nConfusion Matrix: \n")
print(sess.run(tf.confusion_matrix(labels=labels, predictions=predictions)))

===== Logistic Regression on coronary heart disease dataset =====
Training accuracy:	 71.816 %
Testing accuracy:	 72.043 %

Confusion Matrix: 

[[58  8]
 [18  9]]
